In [1]:
import pandas as pd
from sqlalchemy import create_engine
from credentials import user, password, server
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [2]:
engine = create_engine("postgres+psycopg2://"+user+":"+password+"@"\
                       +server+":5432/covid_data")

In [3]:
colleges = pd.read_sql("SELECT * FROM colleges;", con=engine)
counties = pd.read_sql("SELECT * FROM counties;", con=engine)
mask_use = pd.read_sql("SELECT * FROM mask_use;", con=engine)
states = pd.read_sql("SELECT * FROM states;", con=engine)
us_totals = pd.read_sql("SELECT * FROM us_totals;", con=engine)

business = pd.read_csv('../data/businesses.csv')
clusters = pd.read_csv('../data/clusters.csv')
hubnash = pd.read_csv('../data/hubNashville_311_Service_Requests.csv', low_memory=False)
hubnash = hubnash[(hubnash['Subrequest Type'] == 'COVID-19 Violations')]

hubnash = hubnash[hubnash['Latitude'].notna()]
clusters = clusters[clusters['Latitude'].notna()]

In [4]:
business.head()

,address,vicinity,name,type
0,"2440 COUCHVILLE PIKE, Nashville","2440 Couchville Pike, Nashville",Wilson Sporting Goods Co,"['point_of_interest', 'establishment']"
1,"1613 Buchanan St, Nashville","1613 Buchanan Street, Nashville",Coin Cloud Bitcoin ATM,"['atm', 'finance', 'point_of_interest', 'estab..."
2,"1613 Buchanan St, Nashville","1613 Buchanan Street, Nashville",Discounts Mart,"['convenience_store', 'food', 'point_of_intere..."
3,"1613 Buchanan St, Nashville","1613 Buchanan Street, Nashville",Northwest Liquors,"['liquor_store', 'point_of_interest', 'store',..."
4,"306 White Bridge Pike, Nashville","306 White Bridge Pike, Nashville",Suds Up Car Wash of West Nashville,"['car_wash', 'point_of_interest', 'establishme..."


In [5]:
clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.817345
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.754560
5,Tyson Foods,Commercial-Warehouse,4/6/2020,280,36.331015,-86.709977


In [6]:
hubnash.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
61,346122,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/15/2020 11:33:26 AM,04/16/2020 02:55:01 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,928 6th Ave S,NASHVILLE,17.0,37203,36.149040,-86.772273,"(36.1490396, -86.77227289999999)"
95,339374,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/04/2020 06:13:59 PM,04/09/2020 01:07:22 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,500 Gallatin Ave,NASHVILLE,5.0,37206,36.181548,-86.749695,"(36.181548, -86.749695)"
110,343513,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/10/2020 10:40:08 AM,04/14/2020 07:39:09 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,5720 Crossings Blvd,NaN,NaN,NaN,36.039608,-86.646541,"(36.039608, -86.646541)"
113,442996,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/12/2020 02:05:29 AM,08/12/2020 04:03:35 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,7689 Hwy 70 S,NASHVILLE,22.0,37221,36.078933,-86.953400,"(36.07893290000001, -86.9533998)"
118,339423,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,04/04/2020 08:12:27 PM,04/09/2020 02:21:09 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,5310 Mt View Rd,ANTIOCH,32.0,37013,36.048182,-86.648447,"(36.0481819, -86.6484474)"


In [7]:
def dist(lat1, long1, lat2, long2):
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [8]:
def find_nearest(lat, long):
    distances = hubnash.apply(
        lambda row: dist(lat, long, row['Latitude'], row['Longitude']), 
        axis=1)
    return hubnash.loc[distances.idxmin(), 'Address']

clusters['violation_address'] = clusters.apply(
    lambda row: find_nearest(row['Latitude'], row['Longitude']), 
    axis=1)

clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude,violation_address
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655,2201 West End Ave
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583,4710 Charlotte Ave
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.817345,Elmington Ave & Sharondale Dr
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.754560,329 Murfreesboro Pike
5,Tyson Foods,Commercial-Warehouse,4/6/2020,280,36.331015,-86.709977,201 Cartwright St


In [9]:
def find_nearest(lat, long):
    distances = hubnash.apply(
        lambda row: dist(lat, long, row['Latitude'], row['Longitude']), 
        axis=1)
    return hubnash.loc[distances.idxmin(), 'Latitude']

clusters['violation_lat'] = clusters.apply(
    lambda row: find_nearest(row['Latitude'], row['Longitude']), 
    axis=1)

clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude,violation_address,violation_lat
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655,2201 West End Ave,36.144703
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583,4710 Charlotte Ave,36.152444
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.817345,Elmington Ave & Sharondale Dr,36.128018
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.754560,329 Murfreesboro Pike,36.144560
5,Tyson Foods,Commercial-Warehouse,4/6/2020,280,36.331015,-86.709977,201 Cartwright St,36.331373


In [10]:
def find_nearest(lat, long):
    distances = hubnash.apply(
        lambda row: dist(lat, long, row['Latitude'], row['Longitude']), 
        axis=1)
    return hubnash.loc[distances.idxmin(), 'Longitude']

clusters['violation_lon'] = clusters.apply(
    lambda row: find_nearest(row['Latitude'], row['Longitude']), 
    axis=1)

clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude,violation_address,violation_lat,violation_lon
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655,2201 West End Ave,36.144703,-86.802655
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583,4710 Charlotte Ave,36.152444,-86.844583
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.817345,Elmington Ave & Sharondale Dr,36.128018,-86.816838
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.754560,329 Murfreesboro Pike,36.144560,-86.754560
5,Tyson Foods,Commercial-Warehouse,4/6/2020,280,36.331015,-86.709977,201 Cartwright St,36.331373,-86.711044


In [11]:
clusters['violation_distance'] = clusters.apply(
    lambda row: dist(row['Latitude'], row['Longitude'], row['violation_lat'], row['violation_lon']), 
    axis=1) * 0.6213712

clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude,violation_address,violation_lat,violation_lon,violation_distance
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655,2201 West End Ave,36.144703,-86.802655,0.000000
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583,4710 Charlotte Ave,36.152444,-86.844583,0.000000
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.817345,Elmington Ave & Sharondale Dr,36.128018,-86.816838,0.057928
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.754560,329 Murfreesboro Pike,36.144560,-86.754560,0.000159
5,Tyson Foods,Commercial-Warehouse,4/6/2020,280,36.331015,-86.709977,201 Cartwright St,36.331373,-86.711044,0.064339


In [12]:
clusters = clusters.sort_values(by='violation_distance')
clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude,violation_address,violation_lat,violation_lon,violation_distance
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655,2201 West End Ave,36.144703,-86.802655,0.0
58,Miss Kelli's,Bar,10/1/2020,14,36.164369,-86.778515,207 Printers Alley,36.164369,-86.778515,0.0
31,Tootsie's,Bar,6/26/2020,27,36.160769,-86.778210,422 Broadway,36.160769,-86.778210,0.0
29,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161446,-86.775726,221 Broadway,36.161446,-86.775726,0.0
28,Dogwood,Restaurant,6/26/2020,19,36.149811,-86.796336,1907 Division St,36.149811,-86.796336,0.0
